## Environment setup

In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tf_keras_vis.gradcam
from huggingface_hub import hf_hub_download
from PIL import Image
from sklearn.model_selection import train_test_split

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
os.environ["PYTHONHASHSEED"] = "42"

## Utility functions

### Model training and CAM generation

In [ ]:
def download_dataset(n: int = 5000) -> pd.DataFrame:
    """Download facial image dataset from HuggingFace."""
    filepath = hf_hub_download(repo_id="rixmape/utkface", filename="data/train-00000-of-00001.parquet", repo_type="dataset")
    df = pd.read_parquet(filepath)
    return df.sample(n=n, random_state=42) if n > 0 else df


def prepare_data(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray]:
    """Process images and prepare training data."""
    images, labels = [], []
    for _, row in df.iterrows():
        with Image.open(BytesIO(row["image"]["bytes"])) as img:
            img = img.convert("L").resize((48, 48))
            img_array = np.array(img, dtype=np.float32) / 255.0
            img_array = img_array.reshape(48, 48, 1)
            images.append(img_array)
            labels.append(row["gender"])
    return np.array(images), np.array(labels)


def get_test_samples(df: pd.DataFrame, n: int = 16) -> tuple[np.ndarray, np.ndarray]:
    """Get consistent test samples for visualization."""
    test_df = df.sample(n=n, random_state=42)
    images, labels = [], []
    for _, row in test_df.iterrows():
        with Image.open(BytesIO(row["image"]["bytes"])) as img:
            img = img.convert("L").resize((48, 48))
            img_array = np.array(img, dtype=np.float32) / 255.0
            img_array = img_array.reshape(48, 48, 1)
            images.append(img_array)
            labels.append(row["gender"])
    return np.array(images), np.array(labels)


def train_model(model: tf.keras.Model, X: np.ndarray, y: np.ndarray) -> tf.keras.Model:
    """Train model with class balancing and early stopping."""
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    class_counts = np.bincount(y_train)
    total = len(y_train)
    class_weights = {i: total / (len(class_counts) * count) for i, count in enumerate(class_counts)}
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=15, batch_size=32, class_weight=class_weights, callbacks=[early_stopping], verbose=1)
    return model


def generate_cam(model: tf.keras.Model, image: np.ndarray, label: int, target_layer: str) -> np.ndarray:
    """Generate Grad-CAM++ visualization for a single image."""
    modifier_fn = lambda m: setattr(m.layers[-1], "activation", tf.keras.activations.linear)
    score_fn = lambda output: output[0][label]
    visualizer = tf_keras_vis.gradcam.GradcamPlusPlus(model, model_modifier=modifier_fn, clone=True)
    cam = visualizer(score_fn, image[np.newaxis, ...], penultimate_layer=target_layer)[0]
    return cam


def display_grid(results_dict: dict, title: str, filename: str, samples_per_row: int = 8):
    """Display comparison grid of CAMs with minimal design."""
    num_variants = len(results_dict)
    num_samples = samples_per_row

    _, axes = plt.subplots(num_variants, num_samples, figsize=(num_samples * 2, num_variants * 2))

    for i, (variant_name, results) in enumerate(sorted(results_dict.items())):
        for j in range(num_samples):
            ax = axes[i, j] if num_variants > 1 else axes[j]
            result = results[j]

            ax.imshow(result["image"], cmap="gray")
            ax.imshow(result["activation"], cmap="jet", alpha=0.4)
            ax.set_xticks([])
            ax.set_yticks([])

            if j == 0:
                ax.set_ylabel(f"{variant_name}", fontsize=10)

    plt.subplots_adjust(wspace=0.02, hspace=0.02)
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches="tight")
    plt.show()

### CAM granularity metrics

In [ ]:
def calculate_concentration_ratio(cam: np.ndarray, top_percent: float = 10.0) -> float:
    """Calculate what percentage of total activation is contained in top N% pixels."""
    flat_cam = cam.flatten()
    total_activation = np.sum(flat_cam)
    if total_activation == 0:
        return 0.0

    top_k = max(1, int(len(flat_cam) * top_percent / 100))
    sorted_cam = np.sort(flat_cam)[::-1]
    top_activation = np.sum(sorted_cam[:top_k])

    return float(top_activation / total_activation)


def calculate_metrics_for_results(results_dict: dict) -> dict:
    """Calculate concentration ratio metrics for all CAMs in results."""
    metrics = {}
    for variant_name, results in results_dict.items():
        variant_metrics = [calculate_concentration_ratio(r["activation"]) for r in results]
        metrics[variant_name] = {"mean": float(np.mean(variant_metrics)), "std": float(np.std(variant_metrics)), "values": variant_metrics}
    return metrics


def plot_metrics(metrics: dict, title: str, filename: str) -> None:
    """Plot concentration ratio metrics as bar chart with error bars."""
    variant_names = list(metrics.keys())
    means = [metrics[v]["mean"] for v in variant_names]
    stds = [metrics[v]["std"] for v in variant_names]

    plt.figure(figsize=(10, 6))
    bars = plt.bar(variant_names, means, yerr=stds, capsize=5, alpha=0.7)

    for bar, mean in zip(bars, means):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01, f"{mean:.3f}", ha="center", fontsize=9)

    plt.ylim(0, min(1.0, max(means) * 1.2))
    plt.ylabel("Concentration Ratio (higher = more granular)")
    plt.title(f"{title} - Quantitative Analysis")
    plt.tight_layout()
    plt.savefig(f"{filename.split('.')[0]}_metrics.png", dpi=300)
    plt.show()

## Dataset preparation

In [ ]:
df = download_dataset(n=2000)
X, y = prepare_data(df)
test_images, test_labels = get_test_samples(df)

## Experiments

### Architecture depth

In [ ]:
def build_vgg_model(num_blocks: int) -> tf.keras.Sequential:
    """Build VGG-like architecture with variable depth."""
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=(48, 48, 1), name="block1_conv1"))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv2"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block1_pool"))

    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv1"))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv2"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block2_pool"))

    if num_blocks >= 3:
        model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv1"))
        model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv2"))
        model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv3"))
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block3_pool"))

    if num_blocks >= 4:
        model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv1"))
        model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv2"))
        model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv3"))
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block4_pool"))

    if num_blocks >= 5:
        model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv1"))
        model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv2"))
        model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv3"))
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block5_pool"))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model


def get_target_layer_depth(num_blocks: int) -> str:
    """Return target layer name for CAM based on model depth."""
    return {2: "block2_conv2", 3: "block3_conv3", 4: "block4_conv3", 5: "block5_conv3"}[num_blocks]


def run_depth_experiment(X: np.ndarray, y: np.ndarray, test_images: np.ndarray, test_labels: np.ndarray, force_train: bool = False) -> dict:
    """Run experiment to assess how model depth affects CAM granularity."""
    results = {}

    for num_blocks in range(2, 6):
        model_path = f"depth_experiment_blocks{num_blocks}.keras"

        if os.path.exists(model_path) and not force_train:
            print(f"Loading pre-trained model: {model_path}")
            model = tf.keras.models.load_model(model_path)
        else:
            print(f"\nTraining {num_blocks}-block model:")
            model = build_vgg_model(num_blocks)
            model = train_model(model, X, y)
            model.save(model_path)

        target_layer = get_target_layer_depth(num_blocks)
        block_results = []

        for img, label in zip(test_images, test_labels):
            cam = generate_cam(model, img, label, target_layer)
            block_results.append({"image": img[:, :, 0], "activation": cam, "label": label})

        results[f"{num_blocks}-block"] = block_results

    title = "Effect of Architecture Depth on CAM Granularity"
    display_grid(results, title, "depth_experiment_images.png")

    metrics = calculate_metrics_for_results(results)
    plot_metrics(metrics, title, "depth_experiment_metrics.png")

    return {"results": results, "metrics": metrics}


depth_results = run_depth_experiment(X, y, test_images, test_labels)

### Feature map resolution

In [ ]:
def build_model_with_resolution(resolution_type: str) -> tf.keras.Sequential:
    """Build 3-block model with specified feature map resolution."""
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=(48, 48, 1), name="block1_conv1"))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv2"))

    # First pooling layer - skipped in high resolution variant
    if resolution_type != "high":
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block1_pool"))

    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv1"))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv2"))

    # Second pooling layer - present in all variants
    model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block2_pool"))

    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv1"))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv2"))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv3"))

    # Third pooling layer - extra in low resolution variant
    if resolution_type == "low":
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block3_pool_extra"))

    model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block3_pool"))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model


def run_resolution_experiment(X: np.ndarray, y: np.ndarray, test_images: np.ndarray, test_labels: np.ndarray, force_train: bool = False) -> dict:
    """Run experiment to assess how feature map resolution affects CAM granularity."""
    resolutions = {"low": "6x6", "medium": "12x12", "high": "24x24"}
    results = {}

    for resolution_level, feature_size in resolutions.items():
        model_path = f"resolution_experiment_{resolution_level}.keras"

        if os.path.exists(model_path) and not force_train:
            print(f"Loading pre-trained model: {model_path}")
            model = tf.keras.models.load_model(model_path)
        else:
            print(f"\nTraining model with {resolution_level} resolution:")
            model = build_model_with_resolution(resolution_level)
            model = train_model(model, X, y)
            model.save(f"resolution_experiment_{resolution_level}.keras")

        target_layer = "block3_conv3"
        resolution_results = []

        for img, label in zip(test_images, test_labels):
            cam = generate_cam(model, img, label, target_layer)
            resolution_results.append({"image": img[:, :, 0], "activation": cam, "label": label})

        results[f"{resolution_level} ({feature_size})"] = resolution_results

    title = "Effect of Feature Map Resolution on CAM Granularity"
    display_grid(results, title, "resolution_experiment_images.png")

    metrics = calculate_metrics_for_results(results)
    plot_metrics(metrics, title, "resolution_experiment_metrics.png")

    return {"results": results, "metrics": metrics}


resolution_results = run_resolution_experiment(X, y, test_images, test_labels)

### Channel depth vs. spatial information

In [ ]:
def build_model_with_channel_space_tradeoff(variant_type: str) -> tf.keras.Sequential:
    """Build model with tradeoff between channel depth and spatial information."""
    model = tf.keras.Sequential()

    # First block - similar across variants
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=(48, 48, 1), name="block1_conv1"))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv2"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block1_pool"))

    # Second block - varies by variant
    if variant_type == "spatial":
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block2_conv1"))
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block2_conv2"))
    else:  # "channel" variant
        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv1"))
        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv2"))

    model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block2_pool"))

    # Third block - varies by variant
    if variant_type == "spatial":
        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block3_conv1"))
        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block3_conv2"))
        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block3_conv3"))
        # No pooling for spatial variant to preserve spatial information
    else:  # "channel" variant
        model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv1"))
        model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv2"))
        model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv3"))
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block3_pool"))

    # Final layers - common structure
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model


def run_channel_space_experiment(X: np.ndarray, y: np.ndarray, test_images: np.ndarray, test_labels: np.ndarray, force_train: bool = False) -> dict:
    """Run experiment to assess channel depth vs spatial information tradeoff on CAM granularity."""
    variants = ["channel", "spatial"]
    results = {}

    for variant_type in variants:
        model_path = f"channel_space_experiment_{variant_type}.keras"

        if os.path.exists(model_path) and not force_train:
            print(f"Loading pre-trained model: {model_path}")
            model = tf.keras.models.load_model(model_path)
        else:
            print(f"\nTraining model with {variant_type} tradeoff:")
            model = build_model_with_channel_space_tradeoff(variant_type)
            model = train_model(model, X, y)
            model.save(model_path)

        target_layer = "block3_conv3"
        variant_results = []

        for img, label in zip(test_images, test_labels):
            cam = generate_cam(model, img, label, target_layer)
            variant_results.append({"image": img[:, :, 0], "activation": cam, "label": label})

        results[variant_type] = variant_results

    title = "Effect of Channel Depth vs. Spatial Information on CAM Granularity"
    display_grid(results, title, "channel_space_experiment_images.png")

    metrics = calculate_metrics_for_results(results)
    plot_metrics(metrics, title, "channel_space_experiment_metrics.png")

    return {"results": results, "metrics": metrics}


channel_space_results = run_channel_space_experiment(X, y, test_images, test_labels)

### Receptive field

In [ ]:
def build_model_with_receptive_field(field_type: str) -> tf.keras.Sequential:
    """Build model with specified receptive field configuration."""
    model = tf.keras.Sequential()

    # Input layer common to all variants
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=(48, 48, 1), name="block1_conv1"))

    if field_type == "small":
        # Small receptive field: fewer layers, only 3x3 kernels
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv2"))
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block1_pool"))

        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv1"))
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block2_pool"))

    elif field_type == "medium":
        # Medium receptive field: standard configuration similar to Pipeline 1
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv2"))
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block1_pool"))

        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv1"))
        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv2"))
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block2_pool"))

        model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv1"))
        model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv2"))

    else:  # "large" receptive field
        # Large receptive field: mix 3x3 and 5x5 kernels or use dilation
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv2"))
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block1_pool"))

        model.add(tf.keras.layers.Conv2D(128, (5, 5), activation="relu", padding="same", name="block2_conv1"))
        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv2"))
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block2_pool"))

        model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", dilation_rate=(2, 2), name="block3_conv1"))
        model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv2"))

    # Common final layers across all variants
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv3"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block3_pool"))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model


def run_receptive_field_experiment(X: np.ndarray, y: np.ndarray, test_images: np.ndarray, test_labels: np.ndarray, force_train: bool = False) -> dict:
    """Run experiment to assess how receptive field size affects CAM granularity."""
    field_types = ["small", "medium", "large"]
    results = {}

    for field_type in field_types:
        model_path = f"receptive_field_experiment_{field_type}.keras"

        if os.path.exists(model_path) and not force_train:
            print(f"Loading pre-trained model: {model_path}")
            model = tf.keras.models.load_model(model_path)
        else:
            print(f"\nTraining model with {field_type} receptive field:")
            model = build_model_with_receptive_field(field_type)
            model = train_model(model, X, y)
            model.save(model_path)

        target_layer = "block3_conv3"
        field_results = []

        for img, label in zip(test_images, test_labels):
            cam = generate_cam(model, img, label, target_layer)
            field_results.append({"image": img[:, :, 0], "activation": cam, "label": label})

        results[field_type] = field_results

    title = "Effect of Receptive Field Size on CAM Granularity"
    display_grid(results, title, "receptive_field_experiment_images.png")

    metrics = calculate_metrics_for_results(results)
    plot_metrics(metrics, title, "receptive_field_experiment_metrics.png")

    return {"results": results, "metrics": metrics}


receptive_field_results = run_receptive_field_experiment(X, y, test_images, test_labels)

### Pooling strategy

In [ ]:
def build_model_with_pooling(pooling_type: str) -> tf.keras.Sequential:
    """Build 3-block model with specified pooling strategy."""
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=(48, 48, 1), name="block1_conv1"))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv2"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block1_pool"))

    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv1"))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv2"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block2_pool"))

    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv1"))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv2"))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv3"))

    if pooling_type == "max_pooling":
        model.add(tf.keras.layers.MaxPooling2D((2, 2), name="block3_pool"))
        model.add(tf.keras.layers.Flatten())
    elif pooling_type == "avg_pooling":
        model.add(tf.keras.layers.AveragePooling2D((2, 2), name="block3_pool"))
        model.add(tf.keras.layers.Flatten())
    elif pooling_type == "global_avg_pooling":
        model.add(tf.keras.layers.GlobalAveragePooling2D(name="global_avg_pool"))
    elif pooling_type == "global_max_pooling":
        model.add(tf.keras.layers.GlobalMaxPooling2D(name="global_max_pool"))
    elif pooling_type == "flatten":
        model.add(tf.keras.layers.Flatten())
    else:
        raise ValueError(f"Unsupported pooling type: {pooling_type}")

    model.add(tf.keras.layers.Dense(512, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model


def run_pooling_experiment(X: np.ndarray, y: np.ndarray, test_images: np.ndarray, test_labels: np.ndarray, force_train: bool = False) -> dict:
    """Run experiment to assess how pooling strategy affects CAM granularity."""
    pooling_types = ["flatten", "max_pooling", "avg_pooling", "global_avg_pooling", "global_max_pooling"]
    results = {}

    for pooling_type in pooling_types:
        model_path = f"pooling_experiment_{pooling_type}.keras"

        if os.path.exists(model_path) and not force_train:
            print(f"Loading pre-trained model: {model_path}")
            model = tf.keras.models.load_model(model_path)
        else:
            print(f"\nTraining model with {pooling_type}:")
            model = build_model_with_pooling(pooling_type)
            model = train_model(model, X, y)
            model.save(model_path)

        target_layer = "block3_conv3"
        pooling_results = []

        for img, label in zip(test_images, test_labels):
            cam = generate_cam(model, img, label, target_layer)
            pooling_results.append({"image": img[:, :, 0], "activation": cam, "label": label})

        results[pooling_type] = pooling_results

    title = "Effect of Pooling Strategy on CAM Granularity"
    display_grid(results, title, "pooling_experiment_images.png")

    metrics = calculate_metrics_for_results(results)
    plot_metrics(metrics, title, "pooling_experiment_metrics.png")

    return {"results": results, "metrics": metrics}


pooling_results = run_pooling_experiment(X, y, test_images, test_labels)

## Comparative Analysis

In [ ]:
def compare_all_experiments() -> None:
    """Compare quantitative metrics across all experiments."""
    aggregated_metrics = {
        "Depth (2-block)": depth_results["metrics"]["2-block"]["mean"],
        "Depth (5-block)": depth_results["metrics"]["5-block"]["mean"],
        "Resolution (high)": resolution_results["metrics"]["high (24x24)"]["mean"],
        "Resolution (low)": resolution_results["metrics"]["low (6x6)"]["mean"],
        "Channel (channel)": channel_space_results["metrics"]["channel"]["mean"],
        "Channel (spatial)": channel_space_results["metrics"]["spatial"]["mean"],
        "Receptive Field (small)": receptive_field_results["metrics"]["small"]["mean"],
        "Receptive Field (large)": receptive_field_results["metrics"]["large"]["mean"],
        "Pooling (max_pooling)": pooling_results["metrics"]["max_pooling"]["mean"],
        "Pooling (flatten)": pooling_results["metrics"]["flatten"]["mean"],
        "Pooling (global_avg)": pooling_results["metrics"]["global_avg_pooling"]["mean"],
    }

    plt.figure(figsize=(12, 6))
    plt.bar(aggregated_metrics.keys(), aggregated_metrics.values(), alpha=0.7)
    plt.ylabel("Concentration Ratio (higher = more granular)")
    plt.title("Key Factors Affecting CAM Granularity")
    plt.tight_layout()
    plt.savefig("all_experiments_comparison.png", dpi=300)
    plt.show()


compare_all_experiments()